In [0]:
########## Run this code snippet when running for the first time and don't repeat it in future (else it will keep on downloading the same stuffs again and again and
########## result in redundant usage of memory)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
import findspark

os.environ["JAVA_HOME"]   = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]  = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import random
import bisect
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date
spark                      = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
pathname = '/content/monte_carlo_simulations.csv'
df = spark.read.csv(pathname,header=True,inferSchema=True)
df.show(3)

+-----------+----------+--------------+---------+-------------------+
|Account_num|Final_date|Unpaid_Balance|Note_Rate|Current_Delinquency|
+-----------+----------+--------------+---------+-------------------+
|       1001|26-12-2033|          5646|        5|                  0|
|       1002|11-02-2031|          6202|        3|                  0|
|       1003|25-04-2029|          6204|        2|                  0|
+-----------+----------+--------------+---------+-------------------+
only showing top 3 rows



**Step 1:-** Adding a column containing current date so that we can know from when we started calculating the EMIs for the loan given.

In [3]:
df = df.withColumn('Current_Date',lit(str(date.today().strftime("%Y-%m-%d"))))
df.select('Account_num','Current_date').show(2)

+-----------+------------+
|Account_num|Current_date|
+-----------+------------+
|       1001|  2020-01-13|
|       1002|  2020-01-13|
+-----------+------------+
only showing top 2 rows



**Step 2 (Loan Term):-** We now calculate the loan term of the loan which will be the difference between current date and Final date of the loan and it will be in terms of months. 

In [4]:
df = df.withColumn('Months',ceil(months_between(
                                          from_unixtime(unix_timestamp('Final_date', 'dd-mm-yyyy')),
                                          from_unixtime(unix_timestamp('Current_Date', 'yyyy-mm-dd'))
                                          )
                                )
                   )
df.show(5)

+-----------+----------+--------------+---------+-------------------+------------+------+
|Account_num|Final_date|Unpaid_Balance|Note_Rate|Current_Delinquency|Current_Date|Months|
+-----------+----------+--------------+---------+-------------------+------------+------+
|       1001|26-12-2033|          5646|        5|                  0|  2020-01-13|   157|
|       1002|11-02-2031|          6202|        3|                  0|  2020-01-13|   132|
|       1003|25-04-2029|          6204|        2|                  0|  2020-01-13|   109|
|       1004|03-03-2025|          9736|        5|                  0|  2020-01-13|    60|
|       1005|10-05-2025|          8299|        7|                  0|  2020-01-13|    60|
+-----------+----------+--------------+---------+-------------------+------------+------+
only showing top 5 rows



**Step 3:- Numerical value of Delinquency:-** We put the delinquencies to buckets as below <t> 
1.  0 ---- B1
2.  1-99 ---- B2
3.  100-179 --- B3
4.  180+   ---- B4

We will obviously use UDF for this purpose.


In [0]:
def bucketing(val):
  if val==0:
    return 'B1'
  elif (val>=1) and (val <100):
    return 'B2'
  elif (val>=100) and (val<=179):
    return 'B3'
  else:
    return 'B4'

In [0]:
bucketudf = udf(bucketing,StringType())

In [7]:
df = df.withColumn('Buckets',bucketudf(df['Current_Delinquency']))
df.show(3)

+-----------+----------+--------------+---------+-------------------+------------+------+-------+
|Account_num|Final_date|Unpaid_Balance|Note_Rate|Current_Delinquency|Current_Date|Months|Buckets|
+-----------+----------+--------------+---------+-------------------+------------+------+-------+
|       1001|26-12-2033|          5646|        5|                  0|  2020-01-13|   157|     B1|
|       1002|11-02-2031|          6202|        3|                  0|  2020-01-13|   132|     B1|
|       1003|25-04-2029|          6204|        2|                  0|  2020-01-13|   109|     B1|
+-----------+----------+--------------+---------+-------------------+------------+------+-------+
only showing top 3 rows



**Step 4(Perform one step simulation):-** 
1. We assume we have the following two matrices as an input(in real life, it is computed using logistic equations and follows Begg and Gray's assumptions but we will skip them for the sake of simplicity)
2. We simulate the next step of delinquency (one step delinquency performed) for forecasting the delinquency of the next period

In [0]:
class Simulate(object):
  def __init__(self):
    '''
      Static proability and multiplier matrices provided
    '''
    self.prob_matrix = np.array([
                                [0.98,0.02,0.0,0.0],
                                [0.0,0.70,0.30,0.0],
                                [0.0,0.20,0.70,0.10],
                                [0,0,0,1]
                      ])
    self.multiplier  = np.array([
                                [1,0,0,0],
                                [0,3,0,0],
                                [0,2,3,0],
                                [0,0,0,1]
                      ])
    self.buckets     = ['B1','B2','B3','B4']
  def get_vector(self,value):
    '''
      Getting cumulative probability distribution vector corresponding to the current bucket
    '''
    return np.cumsum(dict(zip(self.buckets,[self.prob_matrix[i] for i in range(self.prob_matrix.shape[0])]))[value])
  
  def generate_next_bucket(self,value):
    '''
      Simulating next step delinquency
    '''
    r = random.random()
    return self.buckets[bisect.bisect(self.get_vector(value),r)]
 

In [0]:
inst = Simulate()
delinquency_update = udf(inst.generate_next_bucket,StringType())

In [10]:
df = df.withColumn('Next_Delinquency',delinquency_update(df['Buckets']))
df = df.cache() ###### This is needed because of the lazy operation nature of pyspark module
df.show(6)

+-----------+----------+--------------+---------+-------------------+------------+------+-------+----------------+
|Account_num|Final_date|Unpaid_Balance|Note_Rate|Current_Delinquency|Current_Date|Months|Buckets|Next_Delinquency|
+-----------+----------+--------------+---------+-------------------+------------+------+-------+----------------+
|       1001|26-12-2033|          5646|        5|                  0|  2020-01-13|   157|     B1|              B1|
|       1002|11-02-2031|          6202|        3|                  0|  2020-01-13|   132|     B1|              B1|
|       1003|25-04-2029|          6204|        2|                  0|  2020-01-13|   109|     B1|              B1|
|       1004|03-03-2025|          9736|        5|                  0|  2020-01-13|    60|     B1|              B1|
|       1005|10-05-2025|          8299|        7|                  0|  2020-01-13|    60|     B1|              B1|
|       1006|01-06-2031|          7536|        3|                  0|  2020-01-1

**Step 5 (Perform EMI reduction subject to forecasted delinquency) :-** 
1. We will compute the forecasted EMI to be paid for the next period.
2. The default caused on any account will result in loss.


In [0]:
class EMI_Loss_calculator(Simulate):
  '''
    This module is a child module of the parent module to compute EMI and loss
  '''
  def calculate_loss(self,loan_amount):
    '''
      In ideal scenario, Loss will be computed based on a lot of macro economic factors and also property value
      We simplified the term and assumed that the loss is 10% of the loan amount at the moment. (very strict simplification)
    '''
    return 0.10*float(loan_amount)

  def calculate_emi(self,current_dq,forecasted_dq,loan_amt,note_rate,loan_term):
    ''' 
      We calculate The EMI and loss simultaneously
    '''
    current_dq_ind     = self.buckets.index(current_dq)
    forecasted_dq_ind  = self.buckets.index(forecasted_dq)
    if forecasted_dq !='B4' and current_dq!='B4':
      mult               = self.multiplier[current_dq_ind,forecasted_dq_ind]
      emi                = float(np.ppmt(note_rate/12,1,loan_term/12.0,-1*float(loan_amt)))
      new_upb            = loan_amt-emi
      return [new_upb,emi,0.0,0.0]
    elif forecasted_dq=='B4' and current_dq != 'B4':
      new_upb = loan_amt
      return[float(loan_amt),0.0,self.calculate_loss(loan_amt),0.0]
    else:
      new_upb = loan_amt
      return[float(loan_amt),0.0,self.calculate_loss(loan_amt),self.calculate_loss(loan_amt)]

In [0]:
loss_calc     = EMI_Loss_calculator()
loss_emi_calc = udf(loss_calc.calculate_emi,ArrayType(FloatType()))

In [0]:
df = df.withColumn('Combined_result',loss_emi_calc(df['Buckets'],
                                                   df['Next_Delinquency'],
                                                   df['Unpaid_Balance'],
                                                   df['Note_Rate'],
                                                   df['Months']
                                                   ))
df = df.cache()

In [0]:
df = df.withColumn('Future_unpaid_amt', col('Combined_result').getItem(0))
df = df.withColumn('EMI', col('Combined_result').getItem(1))
df = df.withColumn('Forecasted_Loss', col('Combined_result').getItem(2))
df = df.withColumn('Previous_Loss', col('Combined_result').getItem(3))
df = df.withColumn('New_loss',col('Forecasted_Loss')-col('Previous_Loss'))
df = df.drop('Combined_result')
df = df.cache()

In [15]:
df.show(3)

+-----------+----------+--------------+---------+-------------------+------------+------+-------+----------------+-----------------+---------+---------------+-------------+--------+
|Account_num|Final_date|Unpaid_Balance|Note_Rate|Current_Delinquency|Current_Date|Months|Buckets|Next_Delinquency|Future_unpaid_amt|      EMI|Forecasted_Loss|Previous_Loss|New_loss|
+-----------+----------+--------------+---------+-------------------+------------+------+-------+----------------+-----------------+---------+---------------+-------------+--------+
|       1001|26-12-2033|          5646|        5|                  0|  2020-01-13|   157|     B1|              B1|        5621.0527|24.947412|            0.0|          0.0|     0.0|
|       1002|11-02-2031|          6202|        3|                  0|  2020-01-13|   132|     B1|              B1|        6056.2974| 145.7027|            0.0|          0.0|     0.0|
|       1003|25-04-2029|          6204|        2|                  0|  2020-01-13|   109| 

**Final Step- Summary and results:-** 
1. We will find the aggregate unpaid balances, EMI and loss at the end and beginning of the period
2. We will also see the movement of units from one bucket to another.

In [16]:
print('Distribution table Corresponding to Losses incurred while moving from one bucket to another is :- \n')
df.groupBy(['Buckets',"Next_Delinquency"]).agg({'New_loss':'sum',
                                    'Forecasted_Loss':'sum',
                                    'Previous_Loss':'sum'
                                    }).show()  
print('Distribution of Unpaid balance after next period will be :- \n')
df.groupBy(['Buckets',"Next_Delinquency"]).agg({'Unpaid_Balance':'sum',
                                    'EMI':'sum',
                                    'Future_unpaid_amt':'sum'}).show()

Distribution table Corresponding to Losses incurred while moving from one bucket to another is :- 

+-------+----------------+------------------+--------------------+-----------------+
|Buckets|Next_Delinquency|sum(Previous_Loss)|sum(Forecasted_Loss)|    sum(New_loss)|
+-------+----------------+------------------+--------------------+-----------------+
|     B2|              B2|               0.0|                 0.0|              0.0|
|     B4|              B4| 4590.299987792969|   4590.299987792969|              0.0|
|     B2|              B3|               0.0|                 0.0|              0.0|
|     B3|              B4|               0.0|   541.7000122070312|541.7000122070312|
|     B1|              B2|               0.0|                 0.0|              0.0|
|     B1|              B1|               0.0|                 0.0|              0.0|
|     B3|              B2|               0.0|                 0.0|              0.0|
|     B3|              B3|               0.0|     

In [17]:
print('The inter bucket movement is given by :- ')
df.crosstab('Buckets','Next_Delinquency').show()

The inter bucket movement is given by :- 
+------------------------+----+---+---+---+
|Buckets_Next_Delinquency|  B1| B2| B3| B4|
+------------------------+----+---+---+---+
|                      B2|   0|239|111|  0|
|                      B1|1055| 20|  0|  0|
|                      B4|   0|  0|  0|  6|
|                      B3|   0|  5| 12|  1|
+------------------------+----+---+---+---+



**Bonus Round:-**
Compute the 39 month delinquency chain for each accounts. 
1. We will be forecasting for 39 periods (as per CCAR requirements).
2. The simulations will be done 10 times

In [0]:
class Simulate(object):
  def __init__(self):
    '''
      Static proability and multiplier matrices provided
    '''
    self.prob_matrix = np.array([
                                [0.98,0.02,0.0,0.0],
                                [0.0,0.70,0.30,0.0],
                                [0.0,0.20,0.70,0.10],
                                [0,0,0,1]
                      ])
    self.multiplier  = np.array([
                                [1,0,0,0],
                                [0,3,0,0],
                                [0,2,3,0],
                                [0,0,0,1]
                      ])
    self.buckets     = ['B1','B2','B3','B4']
  def get_vector(self,value):
    '''
      Getting cumulative probability distribution vector corresponding to the current bucket
    '''
    return np.cumsum(dict(zip(self.buckets,[self.prob_matrix[i] for i in range(self.prob_matrix.shape[0])]))[value])
  
  def generate_next_bucket(self,value):
    '''
      Simulating next step delinquency
    '''
    r = random.random()
    return self.buckets[bisect.bisect(self.get_vector(value),r)]

  def generate_entire_path(self,value):
    '''
      Simulates paths with respect to :- simulations and time period
    '''
    
    path_info = []
    for sims in range(10):
      period = 0
      for i in range(39):
        if i == 0:
          previous_dq = 'NIL'
          current_dq  = value
        else:
          period =period+1
          previous_dq = current_dq
          current_dq  = self.generate_next_bucket(previous_dq)
        path_info.append([sims+1,period,previous_dq,current_dq])
    return path_info
 

In [0]:
def bucketing(val):
  if val==0:
    return 'B1'
  elif (val>=1) and (val <100):
    return 'B2'
  elif (val>=100) and (val<=179):
    return 'B3'
  else:
    return 'B4'
bucketudf = udf(bucketing,StringType())

In [0]:
pathname = '/content/monte_carlo_simulations.csv'
df = spark.read.csv(pathname,header=True,inferSchema=True)
df = df.withColumn('Current_Date',lit(str(date.today().strftime("%Y-%m-%d"))))
df = df.withColumn('Months',ceil(months_between(
                                          from_unixtime(unix_timestamp('Final_date', 'dd-mm-yyyy')),
                                          from_unixtime(unix_timestamp('Current_Date', 'yyyy-mm-dd'))
                                          )
                                )
                   )
df = df.withColumn('Buckets',bucketudf(df['Current_Delinquency']))
df = df.cache()

In [0]:
inst = Simulate()
path_gen = udf(inst.generate_entire_path,ArrayType(ArrayType(StringType())))

In [0]:
df = df.withColumn('Outputs',path_gen(df['Buckets']))

In [0]:
df = df.withColumn('Explode',explode('outputs'))
df = df.drop('Outputs')

In [0]:
df=df.withColumn('Simulations', col('Explode').getItem(0))
df=df.withColumn('Time period', col('Explode').getItem(1))
df=df.withColumn('Old_Delinquency', col('Explode').getItem(2))
df=df.withColumn('New_Delinquency', col('Explode').getItem(3))
df = df.drop('Explode','Current_Delinquency','Buckets')
df = df.cache()

In [25]:
df.show(5)

+-----------+----------+--------------+---------+------------+------+-----------+-----------+---------------+---------------+
|Account_num|Final_date|Unpaid_Balance|Note_Rate|Current_Date|Months|Simulations|Time period|Old_Delinquency|New_Delinquency|
+-----------+----------+--------------+---------+------------+------+-----------+-----------+---------------+---------------+
|       1001|26-12-2033|          5646|        5|  2020-01-13|   157|          1|          0|            NIL|             B1|
|       1001|26-12-2033|          5646|        5|  2020-01-13|   157|          1|          1|             B1|             B1|
|       1001|26-12-2033|          5646|        5|  2020-01-13|   157|          1|          2|             B1|             B1|
|       1001|26-12-2033|          5646|        5|  2020-01-13|   157|          1|          3|             B1|             B1|
|       1001|26-12-2033|          5646|        5|  2020-01-13|   157|          1|          4|             B1|         

**Performance analysis** We will be doing some performance analysis like movement from one period to another and also verify central limit theorem as more and more simulations are applied

In [0]:
def calculate_crosstab(sim_num,tp):
  return df.filter((df['Simulations']== sim_num) & (df['Time period']==tp)).crosstab('Old_Delinquency','New_Delinquency').orderBy('Old_Delinquency_New_Delinquency')

In [27]:
print('We consider simulation number 3')
print('The movement of values from t=24 to t=25 is given as :-')
calculate_crosstab(3,24).show()
print('The movement of values from t=25 to t=26 is given as :-')
calculate_crosstab(3,25).show()

We consider simulation number 3
The movement of values from t=24 to t=25 is given as :-
+-------------------------------+---+---+---+---+
|Old_Delinquency_New_Delinquency| B1| B2| B3| B4|
+-------------------------------+---+---+---+---+
|                             B1|652| 12|  0|  0|
|                             B2|  0|116| 61|  0|
|                             B3|  0| 36|126| 16|
|                             B4|  0|  0|  0|430|
+-------------------------------+---+---+---+---+

The movement of values from t=25 to t=26 is given as :-
+-------------------------------+---+---+---+---+
|Old_Delinquency_New_Delinquency| B1| B2| B3| B4|
+-------------------------------+---+---+---+---+
|                             B1|632| 20|  0|  0|
|                             B2|  0|118| 46|  0|
|                             B3|  0| 38|132| 17|
|                             B4|  0|  0|  0|446|
+-------------------------------+---+---+---+---+



In [0]:
def check_clt(time_period_to_check):
  for i in range(1,11,1):
    if i ==1:
      df1 = calculate_crosstab(i,time_period_to_check).toPandas().set_index('Old_Delinquency_New_Delinquency')
    else:
      df2 = calculate_crosstab(i,time_period_to_check).toPandas().set_index('Old_Delinquency_New_Delinquency')
      df1 = df1.add(df2) 
    print('simulation ',i,' completed')
  df1['sum']= df1.sum(axis=1)
  return df1.loc[:,"B1":"B4"].div(df1["sum"], axis=0)

In [29]:
check_clt(25)

simulation  1  completed
simulation  2  completed
simulation  3  completed
simulation  4  completed
simulation  5  completed
simulation  6  completed
simulation  7  completed
simulation  8  completed
simulation  9  completed
simulation  10  completed


,B1,B2,B3,B4
Old_Delinquency_New_Delinquency,,,,
B1,0.978121,0.021879,0.000000,0.000000
B2,0.000000,0.721739,0.278261,0.000000
B3,0.000000,0.201237,0.700956,0.097808
B4,0.000000,0.000000,0.000000,1.000000
